<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 440%; font-weight: 700; text-align: center; color: Plum;">
<br>
寫個無聊「遊戲」來練功
<br>
<br>
<br>
<!-- <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 180%; text-align: center;"> -->

</div>

In [ ]:
%%javascript
// 設定output文字顏色。
document.styleSheets[0].addRule('body', 'color: #87CEFA !important;')

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
遊戲規則
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 隨機產生或多方搜集一個英文單字庫(目前該字庫已有37多萬字)。
* 從字庫中取出每一字檢查是否為`ShiftLex`。
* 所謂`ShiftLex`，是筆者和ChatGPT討論後創出的新詞，意思是：
    * 該字本身是有意義的英文單字。
    * 將每一個字母輪流移至(shift)字首，其餘字母順序不變。所有移動後形成的新字都必須為有效英文單字。
* `ShiftLex`範例(目前暫未找出4個字母以上的`ShiftLex`)：
    * art -> rat -> tar
    * tea -> eta -> ate  
* 夠無聊了吧？依這些功能根本談不上是遊戲，充其量只是練功吧了。倒是可以給比較少寫程式的同學作個step-by-step示範。不過這是急就章下產品，來不及優化，不一定是很好的coding示範。各位請多多給予批評和建議。


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
如何「隨機產生」英文單字？
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 簡單：用[random_word package](https://pypi.org/project/Random-Word/)即可。
* `$ pip install random-word`


In [100]:
from random_word import RandomWords

r = RandomWords()
words = set()
for i in range(9):
    words.add(r.get_random_word())
print(f'{len(words) = }')
print(f'{words}')

len(words) = 9
{'frontispieced', 'yoke', 'zeks', 'stoked', 'verticomental', 'gems', 'biblists', 'mindlessness', 'smore'}


<div style="text-align:center"><img src="https://hackmd.io/_uploads/SyI5hFsS3.png" height="350"/></div>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
不過這樣隨機產生太慢了！
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 直接到該[package的github](https://github.com/vaibhavsingh97/random-word)，將它[整個字庫](https://raw.githubusercontent.com/vaibhavsingh97/random-word/master/random_word/database/words.json)copy出來比較快。
* 不過這個字庫的字偏長且冷僻，筆者又從其他提供單字的網站補充了很幾千個較短的常用單字，經過簡單整理後存入`words_raw.dat`檔。
* 這個檔案尚待進一步去蕪存菁，例如過濾重複、刪除過短的字、刪除帶數字或特殊符號者...等等。
* 整理的code在下面：

In [1]:
# 單字字庫整理-1：刪除重複及全轉為小寫
with open('./words_raw.dat', 'r') as f1:
    words = list(set(f1.read().lower().split('\n')))

# 單字字庫整理-2：最少3個字母、字母沒有完全相同、且沒有數字和特殊符號
# 者才納入最終字庫。
symbols = ('.', ',', '?', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '<', '>', '"', "'", '|', ':', ';', '~', '!', '`', '@', '#', '$', '%', '^', '&', '*', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '=', 'þ')
with open('./words_final.dat', 'w') as f2:
    for index, word in enumerate(words):
        leng_word = len(word)
        if leng_word >= 3 and word != word[0]*leng_word and not [char for char in word if char in symbols]:
            print(f'{index}', end='\r')
            _ = f2.write(f'{word}\n')
            f2.flush()

<div style="text-align:center"><img src="https://hackmd.io/_uploads/HJsUHmqEn.jpg" height="350"/></div>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
資料整理好後，正式幹活...
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 目前面臨的問題是：如何得知某個字母的組合是「有效的英文單字」？ChatGPT API等AI工具一定可以搞定，不過殺雞好像不必用牛刀，而且得付token費。暫不考慮。
* 稍為搜尋一下，找到以下3套Python的模組，大扺可滿足需求：
    1. enchant
    2. nltk
    3. spellchecker
* 也許還有別的更好用模組，待日後很有空時再找好了。目前先試用這3套，看效果如何。

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
1. enchant
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install pyenchant`
* [PyEnchant Introdution](https://pyenchant.github.io/pyenchant/)

In [2]:
import enchant

# 使用前
enchant.list_languages()

['en', 'en_AU', 'en_CA', 'en_GB', 'en_US', 'en_ZA']

<div style="text-align:center"><img src="https://hackmd.io/_uploads/r1CGkKF1p.png" height="350"/></div>

In [5]:
import enchant

spellings = ('en_US', 'en_')   # 'en_'好像是英美spelling通吃。
for spelling in spellings:
    d = enchant.Dict(spelling)   # 先設定使用哪種「英文」
    word1 = 'center'   # American spelling
    word2 = 'centre'   # British spelling
    print(f"{spelling}\t{word1}: {d.check(word1)}\t{word2}: {d.check(word2)}")

en_US	center: True	centre: False
en_	center: True	centre: True


In [27]:
with open('./words_final.dat', 'r') as f:
    words = f.read().split('\n')

words[:5]
words[-5:]

['amblyope', 'capably', 'yorlin', 'eloiners', 'atomiferous']

['ascidioida', 'disappear', 'salutarily', 'reflash', 'chlorophoenicite']

In [28]:
# v1: 直白寫
import enchant

d = enchant.Dict('en_')
shift_lexicons = []
for word in words:
    fitting = True
    for index, alphabet in enumerate(word):
        print(f'{index:,} / {len(words):,}', end='\r')
        this_rotation = f'{alphabet}{word[:index]}{word[index+1:]}'
        if not d.check(this_rotation.lower()):
            fitting = False
            break
    if fitting:
       shift_lexicons.append(word)


print(shift_lexicons)
print(len(shift_lexicons))

['asp', 'art', 'ups', 'eel', 'opp', 'own', 'app', 'wee', 'tea', 'rob', 'add', 'opt', 'amt', 'alp', 'arm', 'int', 'apt']
17


In [29]:
# v2: 改為function，同時加上「最少次數」，讓較多的字符合標準。
import enchant
def is_valid_shift_lex(lexicon: str, min_fittings: int) -> bool:
    d = enchant.Dict('en_')
    fittings = 0
    for index, alphebet in enumerate(lexicon):
        this_rotation = f'{alphebet}{lexicon[:index]}{lexicon[index+1:]}'
        if d.check(this_rotation):  # 是有效單字
            fittings += 1
            if fittings == min_fittings:
                break
    return fittings == min_fittings or fittings == len(lexicon)

shift_lexicons = []
for index, word in enumerate(words):
    print(f'{index:,} / {len(words):,}', end='\r')
    if is_valid_shift_lex(lexicon=word, min_fittings=4):
       shift_lexicons.append(word)

print(shift_lexicons)
print(len(shift_lexicons))

['asp', 'art', 'ups', 'eel', 'opp', 'own', 'app', 'wee', 'tea', 'rob', 'add', 'opt', 'amt', 'alp', 'arm', 'int', 'apt']
17


In [30]:
# 印證某單是否符合min_fittings門檻。
import enchant
def is_valid_shift_lex(lexicon: str, min_fittings: int) -> bool:
    d = enchant.Dict('en_')
    fittings = 0
    for index, alphebet in enumerate(lexicon):
        this_rotation = f'{alphebet}{lexicon[:index]}{lexicon[index+1:]}'
        if d.check(this_rotation):
            print(this_rotation)
            fittings += 1
            if fittings == min_fittings:
                break
    return fittings == min_fittings or fittings == len(lexicon)

word = 'acres'
_ = is_valid_shift_lex(lexicon=word, min_fittings=3)

acres
cares
races


In [10]:
# v3: 改為class
import enchant

class ShiftLex():
    def __init__(self):
        self.d = enchant.Dict('en_')

    def is_valid(self, lexicon: str, min_fittings: int) -> bool:
        d = enchant.Dict('en_')
        fittings = 0
        for index, alphebet in enumerate(lexicon):
            this_rotation = f'{alphebet}{lexicon[:index]}{lexicon[index+1:]}'
            if d.check(this_rotation):
                fittings += 1
                if fittings == min_fittings:
                    break
        return fittings == min_fittings or fittings == len(lexicon)


lexicon = ShiftLex()
shift_lexicons = []
for index, word in enumerate(words):
    print(f'{index:,} / {len(words):,}', end='\r')
    if lexicon.is_valid(lexicon=word, min_fittings=3):
       shift_lexicons.append(word)

print(shift_lexicons)
print(len(shift_lexicons))

['asp', 'veneer', 'rapt', 'acme', 'art', 'manatee', 'acres', 'parses', 'ones', 'appal', 'ups', 'steers', 'eel', 'weer', 'cats', 'wast', 'angers', 'assess', 'eels', 'apps', 'eviler', 'hits', 'watt', 'opp', 'alts', 'arced', 'arrest', 'own', 'mites', 'lumps', 'opts', 'abbe', 'app', 'arcing', 'hips', 'wee', 'tea', 'hears', 'ribbers', 'acre', 'ribbed', 'rob', 'add', 'manatees', 'opt', 'arts', 'abbes', 'alps', 'asps', 'ribbing', 'steer', 'orts', 'watts', 'door', 'east', 'caress', 'aids', 'eats', 'greet', 'hanks', 'cares', 'amt', 'robs', 'arms', 'adds', 'pass', 'ores', 'labs', 'kiss', 'greets', 'laps', 'lats', 'veer', 'avers', 'isms', 'educes', 'huts', 'oops', 'ides', 'earring', 'alp', 'ribber', 'arm', 'windless', 'wees', 'hist', 'ends', 'int', 'apt', 'pots', 'lams', 'doors']
92


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
2. nltk(Natural Language Toolkit)
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install nltk`
* [nltk documentation](https://www.nltk.org/)

In [11]:
from nltk.corpus import words

print(f"{'language' in words.words() = }")
print(f"{'langage' in words.words() = }")

'language' in words.words() = True
'langage' in words.words() = False


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
3. spellchecker
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install pyspellchecker`
* [spellcheckertk documentation](https://pyspellchecker.readthedocs.io/en/latest/)

In [25]:
from spellchecker import SpellChecker

spell = SpellChecker()

print(f"{'language'}: {'language' == spell.correction('language')}")
print(f"{'langage'}: {'langage' == spell.correction('langage')}")

print(f"{spell.correction('alexs')}")


language: True
langage: False
alex


<div style="text-align:center"><img src="https://i.imgur.com/ho0NZ9y.png" height="350"/></div>

In [78]:
lexicons = [('Catilexa', 'catilexa'), ('Taipei', 'taipei'), ('Europe', 'europe'), ('Microsoft', 'microsoft'), ('Linux', 'linux'), ('NATO', 'nato'), ('Christmas', 'christmas')]

leng_lex = 9; leng_result = 12

In [79]:
import enchant

d = enchant.Dict('en_')   # 先設定使用哪種「英文」
for lexicon in lexicons:
    print(f"{lexicon[0]:{leng_lex}}: {str(d.check(lexicon[0])):{leng_result}}{lexicon[1]:{leng_lex}}: {d.check(lexicon[1])}")

Catilexa : False       catilexa : False
Taipei   : True        taipei   : False
Europe   : True        europe   : False
Microsoft: True        microsoft: False
Linux    : True        linux    : False
NATO     : True        nato     : False
Christmas: True        christmas: False


In [80]:
from nltk.corpus import words

for lexicon in lexicons:
    print(f"{lexicon[0]:{leng_lex}}: {str(lexicon[0] in words.words()):{leng_result}}{lexicon[1]:{leng_lex}}: {lexicon[0] in words.words()}")

Catilexa : False       catilexa : False
Taipei   : False       taipei   : False
Europe   : False       europe   : False
Microsoft: False       microsoft: False
Linux    : False       linux    : False
NATO     : False       nato     : False
Christmas: True        christmas: True


In [81]:
from spellchecker import SpellChecker

spell = SpellChecker()
for lexicon in lexicons:
    print(f"{lexicon[0]:{leng_lex}}: {str(lexicon[0] == spell.correction(lexicon[0])):{leng_result}}{lexicon[1]:{leng_lex}}: {lexicon[1] == spell.correction(lexicon[1])}")

Catilexa : False       catilexa : False
Taipei   : True        taipei   : True
Europe   : True        europe   : True
Microsoft: False       microsoft: False
Linux    : False       linux    : False
NATO     : True        nato     : True
Christmas: True        christmas: True


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
結論
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 本「遊戲」非常無趣，不過可借此練一下基本功。
* 如果不用ChatGPT等AI方法，要判斷英文單字，暫時找不到一個非常理想的工具。

<div style="text-align:center"><img src="https://hackmd.io/_uploads/r1mSyb9Ja.png" width="350"/></div>

In [77]:
f"""{f'''{f'{f"{1+1}"*2}'+'3'}'''+'0'}"""

'2230'